# MH Algorithm for Optimal Snake Path

In [4]:
import random
import math
from collections import deque

# Grid and utility definitions
N = 8                               # 8x8 grid
ALL_CELLS = {(r, c) for r in range(N) for c in range(N)}
DIRS = [(1,0), (-1,0), (0,1), (0,-1)]

def neighbors(cell):
    r, c = cell
    for dr, dc in DIRS:
        rr, cc = r+dr, c+dc
        if 0 <= rr < N and 0 <= cc < N:
            yield (rr, cc)

def free_neighbors(path_set, head):
    """Neighbors of head that are inside grid and not yet in the path."""
    return [nbr for nbr in neighbors(head) if nbr not in path_set]

# Target "energy" (favor long paths)
# E(path) = -len(path)  =>  pi(path) ∝ exp(beta * len)
def energy(path):
    return -len(path)

# Proposals (q): small reversible edits
# We mix three moves to improve exploration:
#   1) Extend head by one free neighbor (growth)
#   2) Trim tail by k steps (backtrack)
#   3) Corner flip (local 2-opt-style pivot) to escape traps
# We keep track of forward/reverse proposal counts to form MH ratio.
def propose(path):
    """
    Returns:
        new_path, q_forward, q_reverse
    where q_forward is proposal prob of new|old up to a constant,
    and q_reverse is proposal prob of old|new up to the same constant.
    We use counts of available options so Hastings ratio is meaningful.
    """
    path_deque = deque(path)
    path_set = set(path_deque)
    head = path_deque[-1]

    # Mix of move types
    move_type = random.choices(
        population=["extend", "trim", "flip"],
        weights=[0.55, 0.35, 0.10],
        k=1
    )[0]

    if move_type == "extend":
        opts = free_neighbors(path_set, head)
        if not opts:
            # No extension possible -> return same state (implicit reject)
            return path, 1.0, 1.0
        new_head = random.choice(opts)
        new_path = list(path_deque) + [new_head]

        # forward: choose "extend" (same global weight constant) then 1/|opts|
        qf = max(1, len(opts))    # proportional to number of options (constant factor cancels)
        # reverse: from new state, a valid reverse move is "trim 1" (unique)
        # but to be consistent with our trim move, we assume reverse chooses "trim"
        # among its menu; we approximate reverse options by number of trim magnitudes available.
        # Here, any k in [1 .. len(new_path)-1] is a legal trim (never invalid), so count that.
        qr = max(1, len(new_path)-1)
        return new_path, qf, qr

    if move_type == "trim":
        if len(path_deque) <= 1:
            return path, 1.0, 1.0
        # choose a random trim length k in [1 .. len(path)-1]
        max_k = len(path_deque) - 1
        k = random.randint(1, max_k)
        new_path = list(path_deque)[:len(path_deque)-k]

        # forward: choose one of max_k trims uniformly
        qf = max_k

        # reverse: from the shorter path, potential reverse is an extend of length k,
        # but we only extend by one in our primitive move; to keep a reasonable Hastings
        # ratio, approximate reverse choice count as the number of current free neighbors
        # from the new head (one-step extension possibility count).
        new_head = new_path[-1]
        new_set = set(new_path)
        reverse_opts = free_neighbors(new_set, new_head)
        qr = max(1, len(reverse_opts))
        return new_path, qf, qr

    # Try to "pivot" around head's neighbor to a different free neighbor,
    # replacing the last step if it creates a self-avoiding continuation.
    if move_type == "flip":
        if len(path_deque) < 3:
            return path, 1.0, 1.0
        prev = path_deque[-2]
        head = path_deque[-1]
        # Head's free neighbors except the previous cell and those in path
        cand = [nbr for nbr in neighbors(prev) if nbr != head and nbr not in path_set]
        if not cand:
            return path, 1.0, 1.0
        new_head = random.choice(cand)
        new_path = list(path_deque)
        new_path[-1] = new_head  # flip the last segment to new_head
        # Check still self-avoiding:
        if len(set(new_path)) != len(new_path):
            return path, 1.0, 1.0

        # forward count ~ number of cand options
        qf = max(1, len(cand))
        # reverse count: from new state, how many flips of the new last turn?
        # Approximate similarly:
        new_prev = new_path[-2]
        new_set = set(new_path[:-1])  # all but new head are occupied; new head is unique
        cand_rev = [nbr for nbr in neighbors(new_prev) if nbr != new_head and nbr not in new_set]
        qr = max(1, len(cand_rev))
        return new_path, qf, qr

    # Fallback
    return path, 1.0, 1.0

# Metropolis–Hastings driver
def mh_long_snake(
    steps=200_000,
    beta=1.5,
    restarts=5,
    start_cell=(0,0),
    seed=42
):
    """
    Run multiple MH restarts to search for very long self-avoiding paths
    on an 8x8 grid. Returns (best_path, best_len, history_lengths).
    """
    random.seed(seed)
    best_path_overall = None
    best_len_overall = 0
    history = []

    for r in range(restarts):
        # Start each chain as length-1 path at start_cell
        path = [start_cell]
        best_local = path

        for t in range(steps):
            cand, qf, qr = propose(path)

            # If proposal didn't change (e.g., dead move), just continue
            if cand == path:
                history.append(len(path))
                continue

            # MH acceptance
            dE = energy(cand) - energy(path)           # = ( -len(cand) + len(path) )
            # Target ratio = exp(-beta * dE) = exp(beta * (len(cand) - len(path)))
            target_ratio = math.exp(-beta * dE)
            # Hastings correction using proposal counts (q terms); use qr/qf
            hastings = (qr / qf)
            alpha = min(1.0, target_ratio * hastings)

            if random.random() < alpha:
                path = cand  # accept

            # Track local & global best
            if len(path) > len(best_local):
                best_local = path
            if len(best_local) > best_len_overall:
                best_len_overall = len(best_local)
                best_path_overall = list(best_local)

            # Save a light history of lengths (optional)
            if (t % 500) == 0:
                history.append(len(best_local))

        # Small random restart jitter: move start cell to a random free cell
        start_cell = (random.randrange(N), random.randrange(N))

    return best_path_overall, best_len_overall, history

# Run the search and report
if __name__ == "__main__":
    BEST, LBEST, hist = mh_long_snake(
        steps=500_000,   # increase for more thorough search
        beta=1.8,        # higher beta biases more strongly to longer paths
        restarts=6,
        start_cell=(0,0),
        seed=7
    )

    KNOWN_MAX = N * N   # 64 on 8x8 (Hamiltonian path length)
    print(f"Known theoretical maximum length on {N}x{N}: {KNOWN_MAX}")
    print(f"Best length found by MH: {LBEST}")
    print(f"Gap to maximum: {KNOWN_MAX - LBEST}")

    # Optional: show a tiny ASCII view of the best path order
    if BEST is not None:
        order = {cell: i for i, cell in enumerate(BEST)}
        print("\nBest path order (row,col -> step index):")
        for r in range(N):
            row = []
            for c in range(N):
                row.append(f"{order.get((r,c), -1):2d}")
            print(" ".join(row))

Known theoretical maximum length on 8x8: 64
Best length found by MH: 57
Gap to maximum: 7

Best path order (row,col -> step index):
12 13 14 15 24 25 26 27
11 10 17 16 23 22 -1 28
-1  9 18 19 20 21 30 29
-1  8 43 42 41 40 31 32
 6  7 44 -1 -1 39 38 33
 5  4 45 -1 51 52 37 34
 2  3 46 47 50 53 36 35
 1  0 -1 48 49 54 55 56
